<a href="https://colab.research.google.com/github/CharlesPoletowin/BigDataProject/blob/main/assignment3_motor_vehicle_collisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Install openclean

We use openclean to finish part one of the project. 

In [ ]:
!pip install openclean[full]

# 2 Download and profile dataset

We use socrata from openclean to download the given dataset. 
Code below is from the sample notebook of openclean. 

In [ ]:
import gzip
import os
from openclean.data.load import dataset
from openclean.data.source.socrata import Socrata
from openclean.profiling.dataset import dataset_profile

# API for the motor_vehicle_collision dataset
datafile = "./h9gi-nx95.tsv.gz"
socrata = Socrata().dataset("h9gi-nx95")

# download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        socrata.write(f)

# load the file in memory
ds = dataset(datafile)

# print dataset's profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
CRASH DATE,1840610,0,3422,0.001859,11.677789
CRASH TIME,1840610,0,1440,0.000782,8.928690
BOROUGH,1840610,568574,5,0.000004,2.118312
ZIP CODE,1840610,568796,232,0.000182,7.221106
LATITUDE,1840610,215114,122344,0.075266,15.632018
LONGITUDE,1840610,215114,95967,0.059039,15.342260
LOCATION,1840610,215114,243349,0.149708,16.180344
ON STREET NAME,1840610,376267,15998,0.010925,10.584312
CROSS STREET NAME,1840610,661828,19245,0.016326,11.808140
OFF STREET NAME,1840610,1556712,178991,0.630476,16.906741


# 3 Analyzing dataset columns

We can use dataset_profile from openclean to analyze the columns of 
the given dataset and divide columns into 5 categories. 

## 3.1 Should not to be cleaned (primary key): 
Column COLLISION_ID is the primary key of the dataset, it should not 
be cleaned.  
+ COLLISION_ID

## 3.2 Don't need to be cleaned (already clean): 
Column CRASH DATE and CRASH TIME don't have empty values and their 
existing values are standard according to preview of the dataset. 
Therefore, they don't need to be cleaned. 
+ CRASH DATE
+ CRASH TIME

The following columns are int typed and don't have empty or negative 
values according to the profile and preview of the dataset. Therefore, 
they don't need to be cleaned. 
+ NUMBER OF PEDESTRIANS INJURED
+ NUMBER OF PEDESTRIANS KILLED
+ NUMBER OF CYCLIST INJURED
+ NUMBER OF CYCLIST KILLED
+ NUMBER OF MOTORIST INJURED
+ NUMBER OF MOTORIST KILLED

## 3.3 Need to be cleaned (kind of easy ones): 
Column NUMBER OF PERSONS INJURED and NUMBER OF PERSONS KILLED should 
be equal to the sum of the above NUMBER OF ... INJURED columns and 
NUMBER OF ... KILLED columns, respectively. 
+ NUMBER OF PERSONS INJURED
+ NUMBER OF PERSONS KILLED

Column CONTRIBUTING FACTOR VEHICLE (1-5) and VEHICLE TYPE CODE (1-5) 
should be cleaned for misspellings and unrecognizable values. 
+ CONTRIBUTING FACTOR VEHICLE (1-5)
+ VEHICLE TYPE CODE (1-5)

## 3.4 Can be cleaned (difficult ones): 
The following columns are related to each other. The column LOCATION is 
the combination of the column LATITUDE and column LONGITUDE. The column 
LATITUDE and column LONGITUDE can be used to determine column ZIP CODE 
and column BOROUGH. However, we need extra information to clean 
these columns. 
+ BOROUGH
+ ZIP CODE
+ LATITUDE
+ LONGITUDE
+ LOCATION

## 3.5 Maybe can be cleaned (too difficult ones):
The following columns are also related to column LOCATION, but they cannot 
be easily calculated. Moreover, NYC has lots of streets in the format 
of "W 51 Street", so the misspellings in the columns are also hard to 
be cleaned. 
+ ON STREET NAME
+ CROSS STREET NAME
+ OFF STREET NAME





# 4 Cleaning 
We cleaned all the columns that can be cleaned in this 
chapter starting from the easy ones. 

## 4.1 Need to be cleaned (kind of easy ones):

For the following columns, we cleaned them by filling
empties with sum of other columns. 
+ NUMBER OF PERSONS INJURED  
+ NUMBER OF PERSONS KILLED

In [ ]:
from openclean.function.value.null import is_empty
from openclean.operator.transform.update import update

# cleaning null values 
ds = update(
    ds, 
    ["NUMBER OF PERSONS INJURED", "NUMBER OF PEDESTRIANS INJURED", "NUMBER OF CYCLIST INJURED", "NUMBER OF MOTORIST INJURED"], 
    lambda a,b,c,d: (b+c+d,b,c,d) if is_empty(a) else (a,b,c,d)
    )

ds = update(
    ds, 
    ["NUMBER OF PERSONS KILLED", "NUMBER OF PEDESTRIANS KILLED", "NUMBER OF CYCLIST KILLED", "NUMBER OF MOTORIST KILLED"], 
    lambda a,b,c,d: (b+c+d,b,c,d) if is_empty(a) else (a,b,c,d)
    )

For the following columns, we cleaned them by filling 
empties with "Unspecified" and correcting misspellings using KNN 
clusters. 
+ CONTRIBUTING FACTOR VEHICLE (1-5)

In [ ]:
# cleaning null values and none-strings
for i in range(1, 6): 
  col_name = "CONTRIBUTING FACTOR VEHICLE {}".format(i)

  ds = update(
      ds, 
      col_name, 
      lambda x: "Unspecified" if is_empty(x) or x.isdigit() else x
      )

In [ ]:
from openclean.cluster.knn import knn_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.value.threshold import GreaterThan
from openclean.function.eval.domain import Lookup

# cleaning using cluster and mapping
for i in range(1, 6): 
  col_name = "CONTRIBUTING FACTOR VEHICLE {}".format(i)

  # edit distance cluster
  clusters = knn_clusters(
      values=ds[col_name].unique().tolist(),
      sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.8))
      )

  mapping = {}
  for cluster in clusters: 
    mapping.update(cluster.to_mapping())

  ds = update(
      ds, 
      col_name, 
      Lookup(columns=[col_name], mapping=mapping, default=col_name)
      )

For the following columns, we cleaned them by setting values
in the title format, filling empties with "Unknown", correcting 
misspellings using KNN clusters, and replacing unrecognizable values with 
"Unknown". 
+ VEHICLE TYPE CODE (1-5)

In [ ]:
# cleaning letter case and null
for i in range(1, 6): 
  col_name = "VEHICLE TYPE CODE {}".format(i)
  ds = update(
      ds, 
      col_name, 
      str.title
      )

  ds = update(
      ds, 
      col_name, 
      lambda x: "Unknown" if is_empty(x) or x == "Other" else x
      )

In [ ]:
from openclean.cluster.knn import knn_collision_clusters
from openclean.function.similarity.text import JaroWinklerSimilarity

# cleaning using cluster and mapping
for i in range(1, 6): 
  col_name = "VEHICLE TYPE CODE {}".format(i)

  # edit distance cluster
  clusters = knn_collision_clusters(
           values=ds[col_name].tolist(),
           sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.7))
         )

  mapping = {}
  for cluster in clusters: 
    mapping.update(cluster.to_mapping())

  ds = update(
      ds, 
      col_name, 
      Lookup(columns=[col_name], mapping=mapping, default=col_name)
      )

  # common substr cluster
  clusters = knn_collision_clusters(
           values=ds[col_name].tolist(),
           sim=SimilarityConstraint(func=JaroWinklerSimilarity(), pred=GreaterThan(0.9))
         )

  mapping = {}
  for cluster in clusters: 
    mapping.update(cluster.to_mapping())

  ds = update(
      ds, 
      col_name, 
      Lookup(columns=[col_name], mapping=mapping, default=col_name)
      )

In [ ]:
# select top 50 types and only keep these types
# all other types are regarded as unrecognizable values
top_types = ds["VEHICLE TYPE CODE 1"].value_counts().head(50).keys()

for i in range(1, 6): 
  col_name = "VEHICLE TYPE CODE {}".format(i)

  ds = update(
      ds, 
      col_name, 
      lambda x: "Unknown" if x not in top_types else x
      )

## 4.2 Can be cleaned (difficult ones):

To clean the following columns, we first deleted rows where all 
these columns were empty.
+ BOROUGH  
+ ZIP CODE
+ LATITUDE  
+ LONGITUDE  
+ LOCATION


In [ ]:
from openclean.operator.transform.filter import delete
from openclean.function.eval.base import Col
from openclean.function.eval.logic import And, Or

ds = delete(
      ds, 
      And(Col("BOROUGH")=="", Col("ZIP CODE")=="", Or(Col("LATITUDE")=="", Col("LATITUDE")=="0"), Or(Col("LONGITUDE")=="", Col("LONGITUDE")=="0"))
      )

Then, we introduced a master dataset that contains all borough-zip-location pairs in NYC. We built the dataset ourselves referring to two web pages: 
https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php and https://gist.github.com/erichurst/7882666. By introducing this master dataset, we 
filled empty ZIP CODE and empty BOROUGH using LATITUDE and LONGITUDE. 

P.S. A better solution would be using Google Geocoding API, it can provide 
more accurate borough-zip-location information. However, using the API 
would cost hundreds of dollars. 

In [ ]:
import pandas as pd

# Given location, return a zipcode.
# Reading a borough-zipcode-location list, then filling the zipcode by location data. 
url = "https://raw.githubusercontent.com/CharlesPoletowin/BigDataProject/main/nyc_zipcodes.csv"
df = pd.read_csv(url, index_col=0)
zipcodes = df.values

def location_to_zip(lat, lng, data):
  res = min(data, key = lambda x: abs(x[1] - lat) + abs(x[2] - lng))
  return str(int(res[0]))

ds = update(
    ds, 
    ["ZIP CODE", "LATITUDE", "LONGITUDE"], 
    lambda a,b,c: (location_to_zip(float(b), float(c), zipcodes), b, c) if (is_empty(a) and not is_empty(b) and not is_empty(c)) else (a, b, c)
    )

In [ ]:
# Filling the borough.
df = pd.read_csv(url)
df = df[['BOR', 'ZIP']]
boroughs = df.values

def zip_to_borough(zip, data):
  res = min(data, key = lambda x: abs(x[1] - zip))
  return str(res[0])

ds = update(
    ds, 
    ["BOROUGH", "ZIP CODE"], 
    lambda a,b: (zip_to_borough(int(b), boroughs), b) if (is_empty(a) and not is_empty(b)) else (a, b)
    )

After filling ZIP CODE and BOROUGH using LATITUDE and LONGITUDE, there
were rows with only BOROUGH information. We determined that one 
location information was not enough for a row, so we deleted these rows. 

In [ ]:
# delete null zips
ds = delete(
      ds, 
      Col("ZIP CODE")==""
      )

Lastly, there were rows with BOROUGH and ZIP CODE information but no other
location information, we filled columns LATITUDE, LONGITUDE, and LOCATION with 
"Unknown".

In [ ]:
ds = update(
    ds, 
    ["LATITUDE", "LONGITUDE", "LOCATION"], 
    lambda a,b,c: ("Unknown", "Unknown", "Unknown") if (is_empty(a) or is_empty(b) or is_empty(c)) else (a, b, c)
    )

## 4.3 Maybe can be cleaned (too difficult ones):

For the following columns, correcting misspelling and replacing 
unrecognizable values were difficult so we merely filled the empties 
with "Unknown". 
+ ON STREET NAME  
+ CROSS STREET NAME  
+ OFF STREET NAME  

P.S. A better solution would be using Google Geocoding API, it can 
provide nearby street name information. However, as mentioned above, 
using the API would cost hundreds of dollars.

In [ ]:
# clean null values
ds = update(
    ds, 
    "ON STREET NAME", 
    lambda x: "Unknown" if is_empty(x.strip()) else x.strip().upper()
    )

ds = update(
    ds, 
    "CROSS STREET NAME", 
    lambda x: "Unknown" if is_empty(x.strip()) else x.strip().upper()
    )

ds = update(
    ds, 
    "OFF STREET NAME", 
    lambda x: "Unknown" if is_empty(x.strip()) else x.strip().upper()
    )

# 5 Result
By the end of our cleaning, we filled all the empty values of the 
dataset. We deleted 10% of the dataset for better data quality. We 
enriched the dataset by using master dataset and other columns of the 
original data. We corrected misspellings and replaced unrecognizable 
values using KNN clusters. 

In [ ]:
# print dataset's profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
CRASH DATE,1658177,0,3422,0.002064,11.676122
CRASH TIME,1658177,0,1440,0.000868,8.926433
BOROUGH,1658177,0,5,0.000003,2.135558
ZIP CODE,1658177,0,240,0.000145,7.335288
LATITUDE,1658177,0,122345,0.073783,15.462460
LONGITUDE,1658177,0,95968,0.057876,15.178413
LOCATION,1658177,0,243350,0.146758,15.999979
ON STREET NAME,1658177,0,8505,0.005129,9.001317
CROSS STREET NAME,1658177,0,8839,0.005331,7.920956
OFF STREET NAME,1658177,0,158261,0.095443,3.306691
